In [26]:
%load_ext autoreload
%autoreload 2
import nnsight
nnsight.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.5.0.dev3'

In [ ]:
%pip install -q nnsight flash_attn

In [ ]:
%pip install  git+https://github.com/huggingface/transformers

In [7]:
from transformers import AutoModelForCausalLM
from nnsight import LanguageModel
LLAMA_LIKE_MODELS = [
    # "sbintuitions/tiny-lm-chat",
    # "Maykeye/TinyLLama-v0", 
    # "axolotl-ai-co/gemma-3-34M",
    # "yujiepan/gemma-tiny-random",
    # "yujiepan/llama-3.3-tiny-random",
    # "yujiepan/llama-2-tiny-random",
    # "yujiepan/mistral-tiny-random",
    # "yujiepan/mixtral-8xtiny-random",
    # "yujiepan/deepseek-llm-tiny-random",
    # "yujiepan/phi-3-tiny-random",
    "yujiepan/phi-3.5-moe-tiny-random",
    "yujiepan/llama-4-tiny-random",
]
for model_name in LLAMA_LIKE_MODELS:
    try:
        hf_model = AutoModelForCausalLM.from_pretrained(model_name)
        config = AutoConfig.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_config(config)
        hf_success = True
        LanguageModel(hf_model)
    except Exception as e:
        hf_success = False
        print(e)
    try:
        LanguageModel(model_name)
        nnsight_success = True
    except Exception as e:
        nnsight_success = False
        print(e)
        # raise e
    print(f"{model_name}: hf_success: {hf_success}, nnsight_success: {nnsight_success}")
    if not hf_success or not nnsight_success:
        print(f"{model_name}: hf_success: {hf_success}, nnsight_success: {nnsight_success}")


Unrecognized keys in `rope_scaling` for 'rope_type'='longrope': {'short_mscale', 'long_mscale'}
Unrecognized keys in `rope_scaling` for 'rope_type'='longrope': {'short_mscale', 'long_mscale'}
Unrecognized keys in `rope_scaling` for 'rope_type'='longrope': {'short_mscale', 'long_mscale'}


No module named 'transformers_modules.microsoft.Phi-3'
yujiepan/phi-3.5-moe-tiny-random: hf_success: True, nnsight_success: False
yujiepan/phi-3.5-moe-tiny-random: hf_success: True, nnsight_success: False
'Llama4Config' object has no attribute 'vocab_size'
'Llama4Config' object has no attribute 'vocab_size'
yujiepan/llama-4-tiny-random: hf_success: False, nnsight_success: False
yujiepan/llama-4-tiny-random: hf_success: False, nnsight_success: False


In [8]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-4b-it")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [17]:
get_num_layers(model)

2

In [19]:
from nnterp.nnsight_utils import get_num_layers, skip_layer, skip_layers, get_layer_output, get_layer_input
import torch as th
from nnterp import StandardizedTransformer
from nnsight import LanguageModel
model = LanguageModel("yujiepan/phi-3-tiny-random", device_map="auto", dispatch=True)

prompt = "Hello, world!"
for layer in [0, 1, -1]:
    if layer >= get_num_layers(model):
        break
    # Test skip_layer function (single layer skipping)
    print(f"skipping layer {layer}")
    with model.trace(prompt):
        skip_layer(model, layer)
        single_skip_output = model.lm_head.output.save()

    # Test that skip_layer(2) is equivalent to skip_layers(2, 2)
    with model.trace(prompt):
        skip_layers(model, layer, layer)
        equivalent_skip_output = model.lm_head.output.save()

    assert th.allclose(
        single_skip_output, equivalent_skip_output, atol=1e-5
    ), "skip_layer should be equivalent to skip_layers with same start and end layer"

# Test multiple skip_layer calls vs single skip_layers call
for r in [1, 2, 3]:
    if get_num_layers(model) - r < 1:
        break
    with model.trace(prompt):
        for i in range(get_num_layers(model) - r):
            skip_layer(model, i)
        multiple_single_skips = model.lm_head.output.save()

    with model.trace(prompt):
        skip_layers(model, 0, get_num_layers(model) - 1 - r)
        single_multiple_skip = model.lm_head.output.save()

    assert th.allclose(
        multiple_single_skips, single_multiple_skip, atol=1e-5
    ), "Multiple skip_layer calls should be equivalent to single skip_layers call"

# Test skip_with parameter - use layer 0 output as input to skip layer 3

# Test skip_with parameter in skip_layers
if get_num_layers(model) > 3:
    with model.trace(prompt):
        layer_0_output = get_layer_output(model, 0)
        skip_layers(model, 1, 3, skip_with=layer_0_output)
        skip_layers_with_custom_output = model.lm_head.output.save()

# Test that skip_with=None (default) vs explicit layer input are equivalent
with model.trace(prompt):
    skip_layer(model, 0)  # skip_with=None (default)
    default_skip_output = model.lm_head.output.save()

with model.trace(prompt):
    layer_0_input = get_layer_input(model, 0)
    skip_layer(model, 0, skip_with=layer_0_input)
    explicit_input_skip_output = model.lm_head.output.save()

assert th.allclose(
    default_skip_output, explicit_input_skip_output, atol=1e-5
), "skip_with=None should be equivalent to skip_with=get_layer_input(layer)"
if get_num_layers(model) > 3:
    with model.trace(prompt):
        layer_0_output = get_layer_output(model, 0)
        skip_layer(model, 3, skip_with=layer_0_output)
        skip_with_custom_output = model.lm_head.output.save()
    # Verify that custom skip_with produces different results than default
    assert not th.allclose(
        skip_with_custom_output, default_skip_output
    ), "Using custom skip_with should produce different results than default"


skipping layer 0
skipping layer 1
skipping layer -1


In [1]:
from nnsight.modeling.vllm import VLLM

# NNsight's VLLM wrapper currently supports "device = cuda" and device = "auto"
vllm = VLLM("Maykeye/TinyLLama-v0", device = "auto", dispatch = True) # See supported models: https://docs.vllm.ai/en/v0.6.4.post1/models/supported_models.html


INFO 06-25 12:37:28 [__init__.py:244] Automatically detected platform cuda.


NameError: name 'InterventionGraph' is not defined

In [50]:
print(llama_model.layers[0].source)

                                     * def forward(
                                     0     self,
                                     1     hidden_states: torch.Tensor,
                                     2     attention_mask: Optional[torch.Tensor] = None,
                                     3     position_ids: Optional[torch.LongTensor] = None,
                                     4     past_key_value: Optional[Cache] = None,
                                     5     output_attentions: Optional[bool] = False,
                                     6     use_cache: Optional[bool] = False,
                                     7     cache_position: Optional[torch.LongTensor] = None,
                                     8     position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,  # necessary, but kept here for BC
                                     9     **kwargs: Unpack[FlashAttentionKwargs],
                                    10 ) -> tuple[torch.FloatTensor, O

In [101]:
prompt = "Hello, world!"
for layer in [0, 1, -1]:
    if layer >= get_num_layers(model):
        break
    # Test skip_layer function (single layer skipping)
    with model.trace(prompt):
        skip_layer(model, layer)
        single_skip_output = model.lm_head.output.save()

    # Test that skip_layer(2) is equivalent to skip_layers(2, 2)
    with model.trace(prompt):
        skip_layers(model, layer, layer)
        equivalent_skip_output = model.lm_head.output.save()

    assert th.allclose(
        single_skip_output, equivalent_skip_output, atol=1e-5
    ), "skip_layer should be equivalent to skip_layers with same start and end layer"

# Test multiple skip_layer calls vs single skip_layers call
for r in [1, 2, 3]:
    if get_num_layers(model) - r < 1:
        break
    with model.trace(prompt):
        for i in range(get_num_layers(model) - r):
            skip_layer(model, i)
        multiple_single_skips = model.lm_head.output.save()

    with model.trace(prompt):
        skip_layers(model, 0, get_num_layers(model) - 1 - r)
        single_multiple_skip = model.lm_head.output.save()

    assert th.allclose(
        multiple_single_skips, single_multiple_skip, atol=1e-5
    ), "Multiple skip_layer calls should be equivalent to single skip_layers call"

# Test skip_with parameter - use layer 0 output as input to skip layer 3

# Test skip_with parameter in skip_layers
if get_num_layers(model) > 3:
    with model.trace(prompt):
        layer_0_output = get_layer_output(model, 0)
        skip_layers(model, 2, 4, skip_with=layer_0_output)
        skip_layers_with_custom_output = model.lm_head.output.save()

# Test that skip_with=None (default) vs explicit layer input are equivalent
with model.trace(prompt):
    skip_layer(model, 0)  # skip_with=None (default)
    default_skip_output = model.lm_head.output.save()

with model.trace(prompt):
    layer_0_input = get_layer_input(model, 0)
    skip_layer(model, 0, skip_with=layer_0_input)
    explicit_input_skip_output = model.lm_head.output.save()

assert th.allclose(
    default_skip_output, explicit_input_skip_output, atol=1e-5
), "skip_with=None should be equivalent to skip_with=get_layer_input(layer)"
if get_num_layers(model) > 3:
    with model.trace(prompt):
        layer_0_output = get_layer_output(model, 0)
        skip_layer(model, 3, skip_with=layer_0_output)
        skip_with_custom_output = model.lm_head.output.save()
    # Verify that custom skip_with produces different results than default
    assert not th.allclose(
        skip_with_custom_output, default_skip_output
    ), "Using custom skip_with should produce different results than default"


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/config.json
Model config Gemma2Config {
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "layer_types": [
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attent

eager


loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/tokenizer_config.json
loading file chat_template.jinja from cache at None
Instantiating Gemma2ForCausalLM model under default dtype torch.float32.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in Gemma2ForCausalLM is torch.float32. 

In [ ]:
from nnsight import LanguageModel
model = LanguageModel("google/gemma-2-2b", device_map="auto")
model_config = model.config
model_model_config = model._model.config

def get_attrs(obj):
    return [attr for attr in dir(obj) if not callable(getattr(obj, attr))]

attrs = set(get_attrs(model_config)) | set(get_attrs(model_model_config))

for attr in sorted(attrs):
    val1 = getattr(model_config, attr, None)
    val2 = getattr(model_model_config, attr, None)
    if val1 != val2:
        if isinstance(val1, dict) and isinstance(val2, dict):
            if val1.keys() != val2.keys():
                print(f"Mismatch in attribute keys '{attr}': model.config={val1.keys()}, model._model.config={val2.keys()}")
            else:
                for k, v in val1.items():
                    if v != val2[k]:
                        print(f"Mismatch in attribute '{attr}[{k}]': model.config={v!r}, model._model.config={val2[k]!r}")
        else:
            print(f"Mismatch in attribute '{attr}': model.config={val1!r}, model._model.config={val2!r}")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/config.json
Model config Gemma2Config {
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "layer_types": [
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attent

Mismatch in attribute '__dict__[_attn_implementation_internal]': model.config=None, model._model.config='sdpa'
Mismatch in attribute '__dict__[_attn_implementation_autoset]': model.config=False, model._model.config=True
Mismatch in attribute '_attn_implementation': model.config='eager', model._model.config='sdpa'
Mismatch in attribute '_attn_implementation_autoset': model.config=False, model._model.config=True
Mismatch in attribute '_attn_implementation_internal': model.config=None, model._model.config='sdpa'


In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
cfg = AutoConfig.from_pretrained("google/gemma-2-2b", attn_implementation="eager")
model = AutoModelForCausalLM.from_config(cfg)
print(cfg._attn_implementation)
print(model.config._attn_implementation)
print(model.model.layers[0].self_attn.config._attn_implementation)
model2 = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", device_map="auto", config=cfg)
model2.config
print(model2.config._attn_implementation)
print(model2.model.layers[0].self_attn.config._attn_implementation)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/config.json
Model config Gemma2Config {
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "layer_types": [
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attent

eager
eager
eager


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Gemma2ForCausalLM.

All the weights of Gemma2ForCausalLM were initialized from the model checkpoint at google/gemma-2-2b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Gemma2ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "pad_token_id": 0
}



eager
eager


In [23]:
from nnsight import LanguageModel
model = LanguageModel("yujiepan/opt-tiny-2layers-random")

In [25]:
model.model.decoder.layers

ModuleList(
  (0-1): 2 x OPTDecoderLayer(
    (self_attn): OPTAttention(
      (k_proj): Linear(in_features=8, out_features=8, bias=True)
      (v_proj): Linear(in_features=8, out_features=8, bias=True)
      (q_proj): Linear(in_features=8, out_features=8, bias=True)
      (out_proj): Linear(in_features=8, out_features=8, bias=True)
    )
    (activation_fn): ReLU()
    (self_attn_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (fc1): Linear(in_features=8, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=8, bias=True)
    (final_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  )
)

In [45]:
from nnsight import LanguageModel
model = LanguageModel("yujiepan/opt-tiny-2layers-random", rename={".decoder.layers": "layers", ".foo.decoder":"dec"})
model.model.layers

ModuleList(
  (0-1): 2 x OPTDecoderLayer(
    (self_attn): OPTAttention(
      (k_proj): Linear(in_features=8, out_features=8, bias=True)
      (v_proj): Linear(in_features=8, out_features=8, bias=True)
      (q_proj): Linear(in_features=8, out_features=8, bias=True)
      (out_proj): Linear(in_features=8, out_features=8, bias=True)
    )
    (activation_fn): ReLU()
    (self_attn_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (fc1): Linear(in_features=8, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=8, bias=True)
    (final_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  )
)

In [55]:
type(std._model)

transformers.models.opt.modeling_opt.OPTForCausalLM

In [51]:
from nnterp import StandardizedTransformer
std = StandardizedTransformer("yujiepan/opt-tiny-2layers-random", check_renaming=False)
std

2025-07-02 20:10:51.849 | INFO     | nnterp.standardized_transformer:__init__:219 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 8, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 8)
      (norm/final_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-1): 2 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=8, out_features=8, bias=True)
            (v_proj): Linear(in_features=8, out_features=8, bias=True)
            (q_proj): Linear(in_features=8, out_features=8, bias=True)
            (out_proj): Linear(in_features=8, out_features=8, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=8, out_features=32, bias=True)
          (fc2): Linear(in_features=32, out_features=8, bias=True)
          (norm/final_layer_norm): LayerNorm((8,), eps=1e-05, elementwise_affin

In [52]:
from nnterp.nnsight_utils import get_unembed_norm
get_unembed_norm(std)

LayerNorm((8,), eps=1e-05, elementwise_affine=True)

In [8]:
from nnterp import StandardizedTransformer
StandardizedTransformer("yujiepan/opt-tiny-2layers-random", check_renaming=True)

2025-07-02 20:36:04.334 | INFO     | nnterp.standardized_transformer:__init__:225 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


ValueError: Could not find mlp module in yujiepan/opt-tiny-2layers-random architecture. This means that it was not properly renamed.
Please pass the name of the mlp module to the mlp_rename argument.

In [2]:
from nnterp import StandardizedTransformer
model = StandardizedTransformer("gpt2", check_renaming=True)

2025-07-02 20:34:12.948 | INFO     | nnterp.standardized_transformer:__init__:225 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


In [3]:
type(model.layers)

nnsight.intervention.envoy.Envoy

In [7]:
from nnterp import StandardizedTransformer
from nnterp.nnsight_utils import get_attention
import torch as th
model = StandardizedTransformer("Maykeye/TinyLLama-v0", device_map="auto")
with model.trace("a"):
    print(model.attentions[0].input.shape)
    print(model.mlps[0].input.shape)

2025-07-02 20:35:26.876 | INFO     | nnterp.standardized_transformer:__init__:225 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


torch.Size([1, 2, 64])
torch.Size([1, 2, 64])


In [62]:
from nnterp import StandardizedTransformer
from nnterp.nnsight_utils import get_attention
import torch as th
model = StandardizedTransformer("gpt2", device_map="auto")
model.tokenizer.chat_template
with model.trace("a"):
    print(
    get_attention(model, 0).inputs[1].keys())
    print(get_attention(model, 0).input.shape)
    print(th.allclose(model.layers_output[0], get_attention(model, 1).input))

llama_model = StandardizedTransformer("Maykeye/TinyLLama-v0", device_map="auto")
with llama_model.trace("a"):
    print(
    get_attention(llama_model, 0).inputs[1].keys())
    print(get_attention(llama_model, 0).input.shape)
    layer_output = llama_model.layers_output[0]
    attention_input = get_attention(llama_model, 1).input
    attn_hidden_states = get_attention(llama_model, 1).inputs[1]["hidden_states"]
    print(f"hs vs input: {th.allclose(attn_hidden_states, attention_input)}")
    print(th.allclose(layer_output, attention_input))
    print(layer_output)
    print(attention_input)

dict_keys(['past_key_value', 'cache_position', 'attention_mask', 'head_mask', 'use_cache', 'output_attentions'])
torch.Size([1, 1, 768])
False
dict_keys(['hidden_states', 'attention_mask', 'position_ids', 'past_key_value', 'output_attentions', 'use_cache', 'cache_position', 'position_embeddings'])
torch.Size([1, 2, 64])
hs vs input: True
False
tensor([[[ 3.4593e-01, -5.6658e-01,  4.0431e-01,  4.2073e-01, -1.2793e+00,
          -1.6680e-01,  1.5621e-02, -4.5038e-01,  1.4098e-01,  6.4284e-01,
          -6.5602e-01, -2.1935e-01,  1.2947e+00,  2.8273e-01, -2.5297e-01,
          -1.9284e-02, -1.5790e-01,  1.3371e-01, -6.2846e-01, -2.4550e-01,
           7.0501e-01, -2.3435e-01,  1.3520e-01,  8.9260e-01,  1.8702e-01,
          -4.8103e-01, -2.1904e-01,  1.0528e+00,  9.3141e-01, -4.1581e-01,
           3.0944e-01,  3.2379e-01,  5.5943e-01,  1.5155e-01, -1.0615e+00,
          -3.4208e-01, -4.4287e-01, -5.4429e-01,  3.1906e-01, -4.4383e-01,
           8.3231e-01,  1.6720e-01,  1.7813e-01,  7.29

In [18]:
from nnsight import LanguageModel
model = LanguageModel("gpt2")
with model.trace("hello"):
    print(model.source)

NNsightException: 

Traceback (most recent call last):
  File "/tmp/ipykernel_31106/2023299643.py", line 4, in <module>
    print(model.source)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 974, in __getattr__
    raise AttributeError(f"{self} has no attribute {name}")

AttributeError: GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
 

In [31]:
from nnsight import NNsight
import torch as th
import torch.nn as nn

def add(a, b):
    return a + b

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(10, 10)

    def forward(self, x):
        foo = x + 2  # can't access from source
        foo_func = add(x, 2)  # can access as go through the artificial `add` function
        foo_trap = self.layer(x) + 1  # accesses `self.layer(x)` but not `self.layer(x) + 1`
        return self.layer(x)


model = NNsight(MyModel())
with model.trace(th.randn(10, 10)):
    print(model.source)

                  * def forward(self, x):
                  0     foo = x + 2  # can't access from source
 add_0        ->  1     foo_func = add(x, 2)  # can access as go through the artificial `add` function
 self_layer_0 ->  2     foo_trap = self.layer(x) + 1  # accesses `self.layer(x)` but not `self.layer(x) + 1`
 self_layer_1 ->  3     return self.layer(x)
                  4 


In [39]:
model._model)

transformers.models.bloom.modeling_bloom.BloomForCausalLM

In [46]:
print(StandardizedTransformer("axolotl-ai-co/gemma-3-34M").model.source)

2025-07-02 14:46:41.795 | INFO     | nnterp.standardized_transformer:__init__:197 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


                                         * def forward(
                                         0     self,
                                         1     input_ids: Optional[torch.LongTensor] = None,
                                         2     attention_mask: Optional[torch.Tensor] = None,
                                         3     position_ids: Optional[torch.LongTensor] = None,
                                         4     past_key_values: Optional[Cache] = None,
                                         5     inputs_embeds: Optional[torch.FloatTensor] = None,
                                         6     use_cache: Optional[bool] = None,
                                         7     output_attentions: Optional[bool] = None,
                                         8     output_hidden_states: Optional[bool] = None,
                                         9     cache_position: Optional[torch.LongTensor] = None,
                                        10     **flash_attn_kw

In [14]:
from nnsight import LanguageModel
model = LanguageModel("gpt2")
with model.trace("a"):
    logits = model.output.logits.save()
    layer_out = model.transformer.h[0].output.save()
print(logits.shape)
print(layer_out.shape)

NNsightException: 

Traceback (most recent call last):
  File "/tmp/ipykernel_7654/3324543563.py", line 5, in <module>
    layer_out = model.transformer.h[0].output.save()
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 140, in output
    return self._interleaver.current.request(
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/interleaver.py", line 801, in request
    value = self.send(Events.VALUE, requester)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/interleaver.py", line 785, in send
    raise response

OutOfOrderError: Value was missed for model.transformer.h.0.output.i0. Did you call an Envoy out of order?

In [12]:
model.config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 4,
  "hidden_act": "silu",
  "hidden_size": 64,
  "initializer_range": 0.02,
  "intermediate_size": 256,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 16,
  "num_hidden_layers": 8,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.53.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [27]:
from nnsight import LanguageModel
if "attn_out" in globals():
    del attn_out
model = LanguageModel("Maykeye/TinyLLama-v0")
with model.trace(["hello", "hello the fox is jumping"]):
    logits = model.output.logits.save()
    attn_out = model.model.layers[0].self_attn.source.attention_interface_0.output[0].save()
    print(attn_out.shape)

print(attn_out.shape)

NameError: name 'attn_out' is not defined

In [17]:
from nnterp import StandardizedTransformer
mname = "Maykeye/TinyLLama-v0"
model = StandardizedTransformer(mname, )
with model.trace(["hello", "hello the fox is jumping"]):
    # print(model.model.layers[0].source)
    logits = model.output.logits.save()
    attn_probs = model.attentions[0].source.attention_interface_0.source.nn_functional_dropout_0.output.save()
    #.attention_interface_0.output[1])
    # print([t.shape if t is not None else "None" for t in model.attentions[0].output])
    print(attn_probs.shape)

print(attn_probs.shape)

2025-07-03 11:42:23.156 | INFO     | nnterp.standardized_transformer:__init__:142 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


torch.Size([2, 16, 6, 6])


In [32]:
from nnterp import StandardizedTransformer
mname = "Maykeye/TinyLLama-v0"
# mname = "google/gemma-2-2b"
mname = "bigscience/bigscience-small-testing"
model = StandardizedTransformer(mname, )
with model.trace(["hello", "hello the fox is jumping"], output_attentions=True):
    # print(model.model.layers[0].source)
    print(model.attentions[0].source)#.attention_interface_0.output[1])
    # print([t.shape if t is not None else "None" for t in model.attentions[0].output])

2025-07-02 13:52:49.551 | INFO     | nnterp.standardized_transformer:__init__:197 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


                              * def forward(
                              0     self,
                              1     hidden_states: torch.Tensor,
                              2     residual: torch.Tensor,
                              3     alibi: torch.Tensor,
                              4     attention_mask: torch.Tensor,
                              5     layer_past: Optional[Cache] = None,
                              6     head_mask: Optional[torch.Tensor] = None,
                              7     use_cache: bool = False,
                              8     output_attentions: bool = False,
                              9     cache_position: Optional[torch.LongTensor] = None,
                             10 ):
                             11     batch_size, q_length, _ = hidden_states.shape
 self_query_key_value_0   -> 12     fused_qkv = self.query_key_value(hidden_states)  # [batch_size, seq_length, 3 x hidden_size]
                             13     # 3 x [batch_siz

In [4]:
from nnterp import StandardizedTransformer

model = StandardizedTransformer("Maykeye/TinyLLama-v0", attn_implementation="eager")
# print(model.attentions[0].source)
# print("================"*3)
with model.scan(["hello", "hello the fox is jumping"]):
    print(model.attentions[0].source.attention_interface_0.source.nn_functional_dropout_0.output)
    # print(model.attentions[0].source.attention_interface_0.output[1].shape)
    # print(model.layers_output[0].shape)

FakeTensor(..., device='meta', size=(2, 16, 6, 6), dtype=torch.bfloat16,
           grad_fn=<ToCopyBackward0>)


In [32]:
type(model._model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [34]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [31]:
from nnsight import LanguageModel
model = LanguageModel("gpt2", attn_implementation="eager")
with model.trace("a"):
    print(model.transformer.h[0].attn.source.attention_interface_0.source.module_attn_dropout_0.output)

KeyError: '<nnsight>'

In [15]:
if isinstance("a", (int, float)):
    print("int or float")
else:
    print("not int or float")

not int or float


In [11]:
from nnsight import LanguageModel
model = LanguageModel("yujiepan/mixtral-8xtiny-random", compile=False)
with model.scan("a"):
    pass

W0702 22:53:54.026000 172684 torch/fx/experimental/symbolic_shapes.py:6679] failed during evaluate_expr(u0, hint=None, size_oblivious=False, forcing_spec=False
E0702 22:53:54.027000 172684 torch/fx/experimental/recording.py:299] failed while running evaluate_expr(*(u0, None, False, False), **{})


NNsightException: 

Traceback (most recent call last):
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tracer.execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 487, in execute
    super().execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 331, in execute
    self.model.interleave(interleaver, self.fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/modeling/mixins/meta.py", line 76, in interleave
    return super().interleave(interleaver, fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 705, in interleave
    interleaver(fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/interleaver.py", line 312, in __call__
    fn(*args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nn

In [1]:
from nnterp import StandardizedTransformer
model = StandardizedTransformer("yujiepan/mixtral-8xtiny-random")

with model.trace(["a", "azazza"]):
    print([t.shape  for t in model.mlps[0].output])

2025-07-02 22:38:10.618 | INFO     | nnterp.standardized_transformer:__init__:221 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


[torch.Size([2, 4, 8]), torch.Size([8, 8])]


In [8]:
from IPython.display import Markdown
Markdown(f"""```py
{model.layers[0].source}
```""")

```py
                                     * def forward(
                                     0     self,
                                     1     hidden_states: torch.Tensor,
                                     2     attention_mask: Optional[torch.Tensor] = None,
                                     3     position_ids: Optional[torch.LongTensor] = None,
                                     4     past_key_value: Optional[tuple[torch.Tensor]] = None,
                                     5     output_attentions: Optional[bool] = False,
                                     6     output_router_logits: Optional[bool] = False,
                                     7     use_cache: Optional[bool] = False,
                                     8     cache_position: Optional[torch.LongTensor] = None,
                                     9     position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,  # necessary, but kept here for BC
                                    10     **kwargs: Unpack[FlashAttentionKwargs],
                                    11 ) -> tuple[torch.FloatTensor, Optional[tuple[torch.FloatTensor, torch.FloatTensor]]]:
                                    12     """
                                    13     Args:
                                    14         hidden_states (`torch.FloatTensor`): input to the layer of shape `(batch, seq_len, embed_dim)`
                                    15         attention_mask (`torch.FloatTensor`, *optional*): attention mask of size
                                    16             `(batch, sequence_length)` where padding elements are indicated by 0.
                                    17         past_key_value (`Tuple(torch.FloatTensor)`, *optional*): cached past key and value projection states
                                    18         output_attentions (`bool`, *optional*):
                                    19             Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                                    20             returned tensors for more detail.
                                    21         output_router_logits (`bool`, *optional*):
                                    22             Whether or not to return the logits of all the routers. They are useful for computing the router loss, and
                                    23             should not be returned during inference.
                                    24         use_cache (`bool`, *optional*):
                                    25             If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                                    26             (see `past_key_values`).
                                    27         cache_position (`torch.LongTensor` of shape `(sequence_length)`, *optional*):
                                    28             Indices depicting the position of the input sequence tokens in the sequence.
                                    29         kwargs (`dict`, *optional*):
                                    30             Arbitrary kwargs to be ignored, used for FSDP and other methods that injects code
                                    31             into the model
                                    32     """
                                    33 
                                    34     residual = hidden_states
                                    35 
 self_input_layernorm_0          -> 36     hidden_states = self.input_layernorm(hidden_states)
                                    37 
                                    38     # Self Attention
 self_self_attn_0                -> 39     hidden_states, self_attn_weights = self.self_attn(
                                    40         hidden_states=hidden_states,
                                    41         position_embeddings=position_embeddings,
                                    42         attention_mask=attention_mask,
                                    43         position_ids=position_ids,
                                    44         past_key_value=past_key_value,
                                    45         output_attentions=output_attentions,
                                    46         use_cache=use_cache,
                                    47         cache_position=cache_position,
                                    48         **kwargs,
                                    49     )
                                    50     hidden_states = residual + hidden_states
                                    51 
                                    52     # Fully Connected
                                    53     residual = hidden_states
 self_post_attention_layernorm_0 -> 54     hidden_states = self.post_attention_layernorm(hidden_states)
 self_block_sparse_moe_0         -> 55     hidden_states, router_logits = self.block_sparse_moe(hidden_states)
                                    56     hidden_states = residual + hidden_states
                                    57 
                                    58     outputs = (hidden_states,)
                                    59 
                                    60     if output_attentions:
                                    61         outputs += (self_attn_weights,)
                                    62 
                                    63     if output_router_logits:
                                    64         outputs += (router_logits,)
                                    65 
                                    66     return outputs
                                    67 
```

In [7]:

print(type(model._model.model.layers[0].block_sparse_moe).__mro__)
try:
    with model.scan("a"):
            pass
            print(type(model.mlps_output[0]))
except Exception as exc:
    print(exc)

gpt2 = StandardizedTransformer("gpt2")
with gpt2.scan("a"):
    pass
    print(type(gpt2.mlps_output[0]))

W0702 22:51:06.512000 172684 torch/fx/experimental/symbolic_shapes.py:6679] failed during evaluate_expr(u0, hint=None, size_oblivious=False, forcing_spec=False
E0702 22:51:06.513000 172684 torch/fx/experimental/recording.py:299] failed while running evaluate_expr(*(u0, None, False, False), **{})
2025-07-02 22:51:06.516 | INFO     | nnterp.standardized_transformer:__init__:221 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.


(<class 'transformers.models.mixtral.modeling_mixtral.MixtralSparseMoeBlock'>, <class 'torch.nn.modules.module.Module'>, <class 'object'>)


Traceback (most recent call last):
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tracer.execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 487, in execute
    super().execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 331, in execute
    self.model.interleave(interleaver, self.fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/modeling/mixins/meta.py", line 76, in interleave
    return super().interleave(interleaver, fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 705, in interleave
    interleaver(fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/interv

In [1]:
import transformers
import nnsight
print(transformers.__version__)
print(nnsight.__version__)

4.54.0.dev0
0.5.0.dev3


In [ ]:
from nnsight import LanguageModel; model = LanguageModel("gpt2")
with model.trace("a"):
    pass

In [3]:
from nnsight import LanguageModel
gemma3 = LanguageModel("axolotl-ai-co/gemma-3-34M", device_map="auto", dispatch=True)
with gemma3.generate("Hello, world!", max_length=10):
    print(gemma3.output.logits.save())

tensor([[[ 0.0000, -0.3108, -0.0311,  ..., -0.0029,  0.1946,  0.3862]]],
       device='cuda:1')


NNsightException: 

Traceback (most recent call last):
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tracer.execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 331, in execute
    self.model.interleave(interleaver, self.fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/modeling/mixins/meta.py", line 76, in interleave
    return super().interleave(interleaver, fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 705, in interleave
    interleaver(fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/interleaver.py", line 312, in __call__
    fn(*args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/modeling/language.py", line 145, in __nnsight_generate__
    output = self._model.generate(*args, **kwargs)
  File "/root/.venv/l